In [1]:
from dataset.mnist_dataset import MnistDataset
from utils.sinusoidal_embeddings import get_sinusoidal_embeddings

In [2]:
dataset = MnistDataset(root_dir="dataset/data/")

Found 60000 images


In [3]:
image_tensor, label = dataset[3]

In [4]:
import torch
torch.unique(image_tensor)

tensor([-1.0000, -0.9529, -0.9373, -0.8745, -0.8431, -0.7490, -0.7176, -0.5843,
        -0.5765, -0.4745, -0.4510, -0.4353, -0.4275, -0.4196, -0.4118, -0.3098,
        -0.2784, -0.2235, -0.1686, -0.1059, -0.0667, -0.0588, -0.0039,  0.0039,
         0.0275,  0.0745,  0.1294,  0.2235,  0.2471,  0.2706,  0.2784,  0.4824,
         0.5451,  0.5843,  0.6314,  0.6471,  0.6784,  0.6941,  0.7020,  0.7333,
         0.7804,  0.7961,  0.8118,  0.8431,  0.8510,  0.9059,  0.9137,  0.9373,
         0.9451,  0.9608,  0.9765,  0.9843,  0.9922,  1.0000])

In [5]:
t_single = torch.tensor([1])
emb_single = get_sinusoidal_embeddings(t_single, embedding_dim=32)
print("Single timestep embedding shape:", emb_single.shape)

Single timestep embedding shape: torch.Size([1, 32])


In [6]:
emb_single

tensor([[8.4147e-01, 5.3317e-01, 3.1098e-01, 1.7689e-01, 9.9833e-02, 5.6204e-02,
         3.1618e-02, 1.7782e-02, 9.9998e-03, 5.6234e-03, 3.1623e-03, 1.7783e-03,
         1.0000e-03, 5.6234e-04, 3.1623e-04, 1.7783e-04, 5.4030e-01, 8.4601e-01,
         9.5042e-01, 9.8423e-01, 9.9500e-01, 9.9842e-01, 9.9950e-01, 9.9984e-01,
         9.9995e-01, 9.9998e-01, 9.9999e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00]])